In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import mne

In [2]:
def filter_subject(original, subject):
    return original[original['Subject'] == subject]

def transform_data(data):
    columns_to_drop = ['ItemNum', 'Condition', 'Subject', 'Timestamp', 'TrialNum', 'Cloze', 'Cloze_Balanced', 'Association', 'Association_RC', 'Association_MC', 'Association_weighted', 'RT', 'ACC', 'rcnoun', 'rcverb']
    data = data.drop(columns_to_drop, axis=1)

    return data.transpose()

In [3]:
def create_raw(data, drop_channels = []):
    sfreq = 500  # Sampling frequency of the data

    # Some information about the channels
    ch_names = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'T7', 'C3', 'Cz', 'C4', 'T8', 'TP9', 'CP5', 'CP1', 'CP2', 'CP6', 'TP10', 'P7', 'P3', 'Pz', 'P4', 'P8', 'PO9', 'O1', 'Oz', 'O2', 'PO10', 'VEOG', 'HEOG']
    ch_types = ['eeg'] * (len(ch_names) - 2) + ['eog', 'eog']

    # Create the info structure needed by MNE
    info = mne.create_info(ch_names, sfreq=sfreq, ch_types=ch_types)

    raw = mne.io.RawArray(data, info)

    #set montage
    raw.set_montage('standard_1020', match_alias = True)

    #drop channels specified in drop_channels
    raw.drop_channels(drop_channels)

    return raw

In [4]:
def create_epochs(raw, original):
    condition_mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'rt': 5}
    duration = 1.4
    delta = 1.0 / raw.info["sfreq"]

    events = mne.make_fixed_length_events(raw, id=1, start=0, duration=duration, overlap=0.0)
    events[:, 2] = original[original['Timestamp'] == -200]['Condition'].map(condition_mapping)
    events[:, 0] += 100

    new_events = np.vstack((events, events.copy()))
    new_events[:, 0] += 599
    new_events[:, -1] = 5

    merged_events = []
    for event1, event2 in zip(events, new_events):
        merged_events.append(event1)
        merged_events.append(event2)

    merged_events = np.array(merged_events)

    epochs = mne.Epochs(
        raw,
        merged_events,
        event_id=condition_mapping,
        tmin=-.2,
        tmax=(duration - delta) - .2,
        baseline=None,
        preload=False,
        metadata=mne.epochs.make_metadata(merged_events, event_id=condition_mapping, tmin=-.2, tmax=(duration - delta) - .2, sfreq=500, row_events=['A', 'B', 'C', 'D', 'rt'], keep_first="rt")[0]
    )

    return epochs, merged_events

In [5]:
def create_evoked(epochs):
    evoked_dict = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
    evoked = epochs.average(picks=['Fz', 'Cz', 'Pz'], method="mean", by_event_type=True)
    evoked_dict = {key: evoked[evoked_dict[key]] for key in evoked_dict.keys()}

    return evoked_dict

In [6]:
%matplotlib inline
def plot_evoked(evoked_dict):
    mne.viz.plot_compare_evokeds(evoked_dict, picks=['Fz'], show=True, invert_y=True)
    mne.viz.plot_compare_evokeds(evoked_dict, picks=['Cz'], show=True, invert_y=True)
    mne.viz.plot_compare_evokeds(evoked_dict, picks=['Pz'], show=True, invert_y=True)

In [7]:
def save_fif(path, epochs):
    epochs.save(path, overwrite=True)

In [8]:
original = pd.read_csv('../data/PLOSONE21lmerERP_ObservedData/CAPExp.csv', delimiter=',')

In [9]:
folder_path = '../data/subjects'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

In [10]:
subject_list = list(set(original["Subject"].tolist()))

for subject in subject_list:
    print("Working on subject ", subject)
    original_filtered = filter_subject(original, subject)
    data = transform_data(original_filtered)
    raw = create_raw(data, ['TP9', 'TP10', 'T7', 'T8', 'PO9', 'PO10'])
    epochs, events = create_epochs(raw, original_filtered)
    # evoked_dict = create_evoked(epochs)
    # plot_evoked(evoked_dict)

    epochs.save('../data/subjects/epochs-s' + str(subject).zfill(2) + '-epo.fif', overwrite=True)

Working on subject  1
Creating RawArray with float64 data, n_channels=34, n_times=77700
    Range : 0 ... 77699 =      0.000 ...   155.398 secs
Ready.
Adding metadata with 6 columns
222 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Using data from preloaded Raw for 222 events and 700 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 1 events and 700 original time points ...
Using data from preloaded Raw for 221 events and 700 original time points ...
Working on subject  3
Creating RawArray with float64 data, n_channels=34, n_times=74900
    Range : 0 ... 74899 =      0.000 ...   149.798 secs
Ready.
Adding metadata with 6 columns
214 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 214 events and 700 original time points ...
1 bad epochs dropped
Using data from preloaded Raw for 1 events and 700 original time points ...
Using dat